In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###위도 경도 km 환산 거리 라이브러리 설치 및 필수 라이브러리 및 메소드 로드

In [1]:
!pip3 install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from haversine import haversine
import pandas as pd
import psycopg2
import json
from tqdm import tqdm

In [5]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

##가까운 숙소

In [41]:
attraction = pd.read_csv("/content/drive/MyDrive/yeoreodigm/data_files/09.15_attraction.csv",index_col=0)
attraction.head(3)

,id,title,address,introduction,latitude,longitude,dial_num,type,region1,region2,image_url,score,tag,theme,cluster,more_info
0,1,사려니숲길,제주특별자치도 제주시 조천읍 교래리 산 137-1,"제주 숨은 비경 31, 삼나무 향기에 취하며 걷는 아름답고 청정한 숲길",33.408450,126.639760,064-900-8800,관광지,제주시,조천,https://api.cdn.visitjeju.net/photomng/imgpath...,1276524.0,"자연경관,커플,봄,흐림,걷기/등산,도보,친구","[0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, ...",3,"사려니숲길은 제주의 숨은 비경 31곳 중 하나로, 비자림로를 시작으로 물찻오름과 사..."
1,2,우도(해양도립공원),제주특별자치도 제주시 우도면 삼양고수물길 1,소가 누워있는 형상을 하고 있는 제주의 가장 큰 부속섬,33.519490,126.951090,064-728-1527,관광지,섬 속의 섬,우도,https://api.cdn.visitjeju.net/photomng/imgpath...,1188600.0,"자연경관,맑음,아이,포토스팟","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, ...",10,우도는 소가 누워있는 모양을 닮았다고 해서 일찍부터 소섬 또는 쉐섬으로 불리웠다. ...
2,3,성산일출봉(UNESCO 세계자연유산),제주특별자치도 서귀포시 성산읍 일출로 284-12,"바다위에 우뚝 솟아난 수성화산·유네스코 세계자연유산, 천연기념물 제420호, 올레1코스",33.462147,126.936424,064-783-0959,관광지,서귀포시,성산,https://api.cdn.visitjeju.net/photomng/imgpath...,1027366.0,"부모,자연경관,포토스팟","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, ...",2,현재 성산일출봉은 새로운 일상을 위하여 사회적 거리두기 의무화 조치가 해제되었습니다...


In [42]:
forest_location = tuple(attraction[attraction['id']==1][['latitude','longitude']].values[0])
forest_location

(33.40845, 126.63976)

In [43]:
park = tuple(attraction[attraction['id']==2][['latitude','longitude']].values[0])
park

(33.51949, 126.95109)

In [44]:
distance = haversine(forest_location,park)
distance

31.40845387240905

###숙소데이터 db와 동기화

In [45]:
db = load_db()
cursor=db.cursor()
sql = "select * from accomodation"
cursor.execute(sql)
result = cursor.fetchall()
accomodation = pd.DataFrame(result)

In [46]:
accomodation.columns = ['id','title','address','introduction','dial_num','type','image_url','popularity','tag','latitude','longitude']

In [48]:
accomodation.head(2)

,id,title,address,introduction,dial_num,type,image_url,popularity,tag,latitude,longitude
0,8,캠핑트리펜션,제주특별자치도 제주시 애월읍 광상로 115,애월읍에 위치한 자연이 숨쉬는 통나무 펜션,064-799-7998,펜션,https://api.cdn.visitjeju.net/photomng/imgpath...,43550,"[민박, 숙소, 펜션, 휴양펜션, 바비큐, 수영장, 독채]",33.4108019,126.4331640
1,9,제주가좋아서 펜션,제주특별자치도 제주시 구좌읍 비자림로 1569,"비자림로 인근에 위치해, 편안하고 조용하게 쉬고 갈 수 있는 펜션",064-782-0122,펜션,https://api.cdn.visitjeju.net/photomng/imgpath...,36476,"[숙소, 안전인증민박, 펜션, 휴양펜션, 바비큐, 수영장, 공공와이파이존]",33.4509119,126.7677935


In [49]:
accomodation = accomodation.sort_values(by='id')
accomodation.reset_index(inplace=True,drop=True)

,id,title,address,introduction,dial_num,type,image_url,popularity,tag,latitude,longitude
0,1,제주조천스위스마을게스트하우스,제주특별자치도 제주시 조천읍 함와로 566-27,조천읍 와산리에 위치한 농촌 관광 공동체 마을,064-744-6060,게스트하우스,https://api.cdn.visitjeju.net/photomng/imgpath...,97453,"[게스트하우스, 숙소]",33.4826200,126.6834800
1,2,해비치호텔앤드리조트,제주특별자치도 서귀포시 표선면 민속해안로 537,제주의 푸른 바다와 자연을 만끽할 수 있는 표선에 위치한 호텔&리조트,064-780-8100,호텔/콘도,https://api.cdn.visitjeju.net/photomng/imgpath...,69809,"[호텔, 숙소, 양식레스토랑, 가족호텔, 주차장, 수영장, 리조트, 공공와이파이존,...",33.3234500,126.8447800
2,3,나미송 머무는 곳,제주특별자치도 제주시 애월읍 평화로 2476,자연과 사람의 서정이 함께 흐르는 아름다운 민박집,010-8307-0531,펜션,https://api.cdn.visitjeju.net/photomng/imgpath...,64177,"[민박, 고향민박, 정원, 주방기구, 자연경관, 독채]",33.4108019,126.4331640
3,4,신라호텔제주,제주도 서귀포시 중문관광로 72번길 75,"파스텔 톤의 인테리어, 세계 유명 작가들의 예술작품과 아열대 정원이 어우러진 리조트 호텔",064-735-5114,호텔/콘도,https://api.cdn.visitjeju.net/photomng/imgpath...,47206,"[호텔, 게스트하우스, 숙소, 양식레스토랑, 주차장, 수영장, 공공와이파이존, 관광호텔]",33.2475600,126.4081200
4,5,롯데호텔 제주,제주특별자치도 서귀포시 중문관광로72번길 35,사계절 온수풀과 이국적인 풍경이 매혹적인 리조트형 호텔,064-731-1000,호텔/콘도,https://api.cdn.visitjeju.net/photomng/imgpath...,47113,"[호텔, 숙소, 조식, 양식레스토랑, 가족호텔, 주차장, 자연경관, 수영장, 리조트...",33.2484577,126.4105487
...,...,...,...,...,...,...,...,...,...,...,...
917,918,유창민박,제주특별자치도 제주시 추자면 추자로 46-1,공연장과 버스정류장 바로 앞 위치,064-742-3555,민박,https://api.cdn.visitjeju.net/photomng/imgpath...,424,"[민박, 숙소, 추자도, 주차장, 농어촌민박]",33.9642903,126.2929862
918,919,더오라,제주특별자치도 제주시 동성길 36-1 3층,제주 시내에 오소록이 숨어있는 농어촌민박 더오라스테이,010-5505-2928,펜션,https://api.cdn.visitjeju.net/photomng/imgpath...,418,"[조식 포함, 숙소, 공공와이파이존]",33.4968978,126.5095169
919,920,보로미 민박,제주특별자치도 제주시 애월읍 고성서3길 20-1,가족이 머물기에 적합한 넓은 숙소입니다. 편안한 휴식을 보낼 수 있습니다.,010-4690-1268,민박,https://api.cdn.visitjeju.net/photomng/imgpath...,414,[숙소],33.4652669,126.4118502
920,921,밀물민박,제주특별자치도 제주시 추자면 추자로 106-1,바다뷰가 보이는 민박,010-9688-2359,민박,https://api.cdn.visitjeju.net/photomng/imgpath...,398,"[민박, 숙소, 힐링, 추자도, 농어촌민박, 공공와이파이존]",33.9567036,126.2969646


In [51]:
accomodation.head(2)

,id,title,address,introduction,dial_num,type,image_url,popularity,tag,latitude,longitude
0,1,제주조천스위스마을게스트하우스,제주특별자치도 제주시 조천읍 함와로 566-27,조천읍 와산리에 위치한 농촌 관광 공동체 마을,064-744-6060,게스트하우스,https://api.cdn.visitjeju.net/photomng/imgpath...,97453,"[게스트하우스, 숙소]",33.4826200,126.6834800
1,2,해비치호텔앤드리조트,제주특별자치도 서귀포시 표선면 민속해안로 537,제주의 푸른 바다와 자연을 만끽할 수 있는 표선에 위치한 호텔&리조트,064-780-8100,호텔/콘도,https://api.cdn.visitjeju.net/photomng/imgpath...,69809,"[호텔, 숙소, 양식레스토랑, 가족호텔, 주차장, 수영장, 리조트, 공공와이파이존,...",33.3234500,126.8447800


In [73]:
accomodation.to_csv("/content/drive/MyDrive/yeoreodigm/data_files/09.29_accomodation.csv")

In [53]:
accomodation['latitude'] = accomodation['latitude'].apply(lambda x: float(x))
accomodation['longitude'] = accomodation['longitude'].apply(lambda x: float(x))

###분류 이전에 테스트

In [54]:
save_entire_near_accoms = []
cnt=0
for now in attraction.values:    
  now_title = now[1]
  now_latitude = now[4]
  now_longitude = now[5]
  now_location = (now_latitude,now_longitude)
  print('now loc : ',now_title)


  for accom in accomodation.values:
    accom_id = accom[0]
    accom_title = accom[1]
    accom_latitude = accom[9]
    accom_longitude = accom[10]
    accom_type = accom[5]
    if accom_type =='기타' or accom_type == '삭제하자':
      print(accom_type)
      continue
    accom_location = (accom_latitude,accom_longitude)
    #print(accom_location)
    
    distance = haversine(now_location,accom_location)
    # print(distance)
    if distance<=5.0:
      #now에서 가까운 숙소니까 저장하기 해당하는 숙소 타입의 index를 찾아서 id 추가
      print('사려니숲길과 가까운 곳',accom_id,accom_title)
  break


now loc :  사려니숲길
사려니숲길과 가까운 곳 16 한화리조트 제주
사려니숲길과 가까운 곳 210 제주베스트힐 펜션&글램핑
사려니숲길과 가까운 곳 260 더갤러리펜션
삭제하자
기타


###가까운 location 찾기

펜션

민박

모텔

게스트하우스

호텔/콘도

기타

캠핑

###실제 분류작업 (가까운 숙소)

In [55]:
type_indicator = {'펜션':0, '민박':1, '모텔':2, '게스트하우스':3, '호텔/콘도':4, '캠핑':5}

In [56]:
save_entire_near_accoms = []
cnt=0
for now in attraction.values:    
  now_latitude = now[4]
  now_longitude = now[5]
  now_location = (now_latitude,now_longitude)
  #print('now loc : ',now_location)
  save_near_accoms = [[] for _ in range(6)]

  for accom in accomodation.values:
    accom_id = accom[0]
    accom_title = accom[1]
    accom_latitude = accom[9]
    accom_longitude = accom[10]
    accom_type = accom[5]
    if accom_type =='기타' or accom_type == '삭제하자':
      #print(accom_type)
      continue
    accom_location = (accom_latitude,accom_longitude)
    #print(accom_location)
    
    distance = haversine(now_location,accom_location)
    # print(distance)
    if distance<=5.0:
      #now에서 가까운 숙소니까 저장하기 해당하는 숙소 타입의 index를 찾아서 id 추가
      save_near_accoms[type_indicator[accom_type]].append(accom_id)

  save_entire_near_accoms.append(save_near_accoms)

In [99]:
save_entire_near_accoms

[]

In [57]:
entire_near_accoms_df = pd.DataFrame(save_entire_near_accoms)

In [58]:
entire_near_accoms_df

,0,1,2,3,4,5
0,"[210, 260]",[],[],[],[16],[]
1,"[48, 84, 109, 192, 202, 253, 381, 412, 433, 45...","[312, 531, 587, 644, 651, 664, 679]",[],"[186, 249, 504, 554, 755]",[],[]
2,"[84, 109, 112, 185, 236, 253, 327, 349, 381, 4...","[367, 465, 679]",[149],"[22, 29, 159, 178, 181, 249, 340, 354, 362, 42...","[7, 34, 38, 40, 43, 50, 51, 107, 317, 397, 619...",[]
3,"[141, 205, 220, 334, 345, 547, 707, 749, 811, ...",[719],[],"[255, 287, 500, 609, 613, 802]",[],[]
4,"[26, 55, 72, 104, 113, 188, 250, 267, 331, 350...","[589, 660]",[],"[45, 92, 127, 133, 138, 316, 460, 509, 550, 63...","[21, 71, 135, 136, 140, 142, 151, 161, 164, 17...",[]
...,...,...,...,...,...,...
1084,[],[],[],[],[],[]
1085,"[48, 84, 109, 112, 185, 192, 236, 253, 349, 38...","[312, 367, 531, 587, 644, 651, 664, 679]",[149],"[22, 159, 186, 249, 427, 504, 554, 607, 755, 906]","[34, 43, 50, 51, 317, 397, 714]",[]
1086,"[11, 27, 94, 96, 160, 162, 184, 202, 220, 263,...","[119, 227, 479, 505, 563, 765]",[],"[35, 49, 53, 64, 116, 255, 288, 475, 500, 515,...",[],[]
1087,"[52, 66, 78, 213, 222, 277, 289, 291, 314, 376...","[551, 585, 603]",[],"[204, 472, 572]","[4, 5, 12, 30, 31, 46, 75, 157, 173, 179, 191,...",[]


###기존 attraction과 Merge 작업

In [59]:
merge_attraction = pd.merge(left=attraction,right=entire_near_accoms_df,left_index=True,right_index=True,how='inner')

In [60]:
merge_attraction.head(3)

,id,title,address,introduction,latitude,longitude,dial_num,type,region1,region2,...,tag,theme,cluster,more_info,0,1,2,3,4,5
0,1,사려니숲길,제주특별자치도 제주시 조천읍 교래리 산 137-1,"제주 숨은 비경 31, 삼나무 향기에 취하며 걷는 아름답고 청정한 숲길",33.408450,126.639760,064-900-8800,관광지,제주시,조천,...,"자연경관,커플,봄,흐림,걷기/등산,도보,친구","[0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, ...",3,"사려니숲길은 제주의 숨은 비경 31곳 중 하나로, 비자림로를 시작으로 물찻오름과 사...","[210, 260]",[],[],[],[16],[]
1,2,우도(해양도립공원),제주특별자치도 제주시 우도면 삼양고수물길 1,소가 누워있는 형상을 하고 있는 제주의 가장 큰 부속섬,33.519490,126.951090,064-728-1527,관광지,섬 속의 섬,우도,...,"자연경관,맑음,아이,포토스팟","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, ...",10,우도는 소가 누워있는 모양을 닮았다고 해서 일찍부터 소섬 또는 쉐섬으로 불리웠다. ...,"[48, 84, 109, 192, 202, 253, 381, 412, 433, 45...","[312, 531, 587, 644, 651, 664, 679]",[],"[186, 249, 504, 554, 755]",[],[]
2,3,성산일출봉(UNESCO 세계자연유산),제주특별자치도 서귀포시 성산읍 일출로 284-12,"바다위에 우뚝 솟아난 수성화산·유네스코 세계자연유산, 천연기념물 제420호, 올레1코스",33.462147,126.936424,064-783-0959,관광지,서귀포시,성산,...,"부모,자연경관,포토스팟","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, ...",2,현재 성산일출봉은 새로운 일상을 위하여 사회적 거리두기 의무화 조치가 해제되었습니다...,"[84, 109, 112, 185, 236, 253, 327, 349, 381, 4...","[367, 465, 679]",[149],"[22, 29, 159, 178, 181, 249, 340, 354, 362, 42...","[7, 34, 38, 40, 43, 50, 51, 107, 317, 397, 619...",[]


In [108]:
#type_indicator = {'펜션':0, '민박':1, '모텔':2, '게스트하우스':3, '호텔/콘도':4, '캠핑':5}

In [61]:
merge_attraction.rename(columns = {0:'near_pension',1:'near_minbak',2:'near_motel',3:'near_guestHouse',4:'near_hotel',5:'near_camping'},inplace=True)

In [63]:
merge_attraction.head(2)

,id,title,address,introduction,latitude,longitude,dial_num,type,region1,region2,...,tag,theme,cluster,more_info,near_pension,near_minbak,near_motel,near_guestHouse,near_hotel,near_camping
0,1,사려니숲길,제주특별자치도 제주시 조천읍 교래리 산 137-1,"제주 숨은 비경 31, 삼나무 향기에 취하며 걷는 아름답고 청정한 숲길",33.40845,126.63976,064-900-8800,관광지,제주시,조천,...,"자연경관,커플,봄,흐림,걷기/등산,도보,친구","[0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, ...",3,"사려니숲길은 제주의 숨은 비경 31곳 중 하나로, 비자림로를 시작으로 물찻오름과 사...","[210, 260]",[],[],[],[16],[]
1,2,우도(해양도립공원),제주특별자치도 제주시 우도면 삼양고수물길 1,소가 누워있는 형상을 하고 있는 제주의 가장 큰 부속섬,33.51949,126.95109,064-728-1527,관광지,섬 속의 섬,우도,...,"자연경관,맑음,아이,포토스팟","[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, ...",10,우도는 소가 누워있는 모양을 닮았다고 해서 일찍부터 소섬 또는 쉐섬으로 불리웠다. ...,"[48, 84, 109, 192, 202, 253, 381, 412, 433, 45...","[312, 531, 587, 644, 651, 664, 679]",[],"[186, 249, 504, 554, 755]",[],[]


##가까운 맛집

###맛집 데이터 db와 동기화

In [65]:
db = load_db()
cursor=db.cursor()
sql = "select * from restaurant"
cursor.execute(sql)
result = cursor.fetchall()
food = pd.DataFrame(result)

In [66]:
food.head(3)

,0,1,2,3,4,5,6,7,8,9,10
0,1443,나라돈까스,제주특별자치도 제주시 절물1길 23,외도일동에 위치한 돈까스 전문점 나라왕돈까스는 상호명 답게 푸짐한 양은 물론이며 모...,064-711-0221,일식,https://api.cdn.visitjeju.net/photomng/imgpath...,10,"[식당, 음식]",33.4907957,126.4317720
1,886,내담,제주특별자치도 서귀포시 예래로 296-1,하예동 내담 아구찜 대구탕 전문점,064-738-2839,한식,https://api.cdn.visitjeju.net/photomng/imgpath...,2538,"[한식, 음식]",33.2414550,126.3830900
2,1202,동우통닭,제주특별자치도 제주시 도남로 76,도심 속 시장통닭,064-757-9352,한식,https://api.cdn.visitjeju.net/photomng/imgpath...,1196,"[음식, 치킨]",33.4936925,126.5279667


In [67]:
food.columns = ['id','title','address','introduction','dial_num','type','image_url','popularity','tag','latitude','longitude']
food.sort_values(by='id',inplace=True)
food.reset_index(inplace=True,drop=True)

In [68]:
food.head(3)

,id,title,address,introduction,dial_num,type,image_url,popularity,tag,latitude,longitude
0,1,제주김만복(만복이네김밥집),제주특별자치도 제주시 오라로 41,"전복내장으로 만든 김밥, 심플한 메뉴로 많은 여행객들에게 사랑받는 곳",064-759-8582,한식,https://api.cdn.visitjeju.net/photomng/imgpath...,144324,"[음식, 라면, 해물라면]",33.4970489,126.5087309
1,2,봄날카페,제주특별자치도 제주시 애월읍 애월로1길 25,드라마 '맨도롱 또똣'의 주인공이 일하던 카페,064-799-4999,카페/베이커리,https://api.cdn.visitjeju.net/photomng/imgpath...,121065,"[아메리카노, 카페모카, 바닐라라떼, 카푸치노, 음료, 카페, 커피, 음식, 카페라떼]",33.4624600,126.3095900
2,3,올래국수,제주특별자치도 제주시 귀아랑길 24 (연동),제주 향토 음식인 고기국수 전문점이다. 2016 KOREAT JEJU TOP 10 선정,064-742-7355,한식,https://api.cdn.visitjeju.net/photomng/imgpath...,119913,"[한식, 음식, 향토음식, 고기국수]",33.4915000,126.4973000


In [12]:
food.to_csv('/content/drive/MyDrive/yeoreodigm/data_files/09.29_restaurant.csv')

In [69]:
food['latitude'] = food['latitude'].apply(lambda x: float(x))
food['longitude'] = food['longitude'].apply(lambda x: float(x))

In [71]:
for tmp in food.values:
  print(tmp)
  break

[1 '제주김만복(만복이네김밥집)' '제주특별자치도 제주시 오라로 41'
 '전복내장으로 만든 김밥, 심플한 메뉴로 많은 여행객들에게 사랑받는 곳' '064-759-8582' '한식'
 'https://api.cdn.visitjeju.net/photomng/imgpath/201804/30/dbd4e45a-3b53-444c-b0dd-740840f9d749.jpg'
 144324 list(['음식', '라면', '해물라면']) 33.4970489 126.5087309]


###분류 이전에 테스트

In [74]:
save_entire_near_foods = []
cnt=0
for now in attraction.values:    
  now_title = now[1]
  now_latitude = now[4]
  now_longitude = now[5]
  now_location = (now_latitude,now_longitude)
  print('now loc : ',now_title)


  for f in food.values:
    food_id = f[0]
    food_title = f[1]
    food_latitude = f[9]
    food_longitude = f[10]
    food_type = f[5]
    if food_type =='폐업':
      continue
    food_location = (food_latitude,food_longitude)
    #print(accom_location)
    
    distance = haversine(now_location,food_location)
    # print(distance)
    if distance<=5.0:
      #now에서 가까운 숙소니까 저장하기 해당하는 숙소 타입의 index를 찾아서 id 추가
      print(now_title,'와(과) 가까운 곳',food_id,food_title)
  cnt+=1
  if cnt==2:
    break


now loc :  사려니숲길
사려니숲길 와(과) 가까운 곳 66 교래손칼국수
사려니숲길 와(과) 가까운 곳 191 2093하우스
사려니숲길 와(과) 가까운 곳 197 더로맨틱
사려니숲길 와(과) 가까운 곳 248 교래흑돼지
사려니숲길 와(과) 가까운 곳 303 손맛촌
사려니숲길 와(과) 가까운 곳 308 성미가든
사려니숲길 와(과) 가까운 곳 392 안다미로
사려니숲길 와(과) 가까운 곳 499 토계정
사려니숲길 와(과) 가까운 곳 592 금보가든
사려니숲길 와(과) 가까운 곳 638 라포레사려니
사려니숲길 와(과) 가까운 곳 643 카페스노우빈
사려니숲길 와(과) 가까운 곳 748 카페갤러리
사려니숲길 와(과) 가까운 곳 825 정봉자커피 봉개점
사려니숲길 와(과) 가까운 곳 1123 앙뚜아네트 삼다수마을점
사려니숲길 와(과) 가까운 곳 1346 돌카롱 사려니숲길점
사려니숲길 와(과) 가까운 곳 1350 산굼부리카페
사려니숲길 와(과) 가까운 곳 1426 절물자연휴양림휴게소
now loc :  우도(해양도립공원)
우도(해양도립공원) 와(과) 가까운 곳 34 하하호호(HAHAHOHO)
우도(해양도립공원) 와(과) 가까운 곳 54 섬소나이
우도(해양도립공원) 와(과) 가까운 곳 117 블랑로쉐
우도(해양도립공원) 와(과) 가까운 곳 134 산도롱맨도롱
우도(해양도립공원) 와(과) 가까운 곳 137 우도 풍원
우도(해양도립공원) 와(과) 가까운 곳 152 봉끄랑
우도(해양도립공원) 와(과) 가까운 곳 176 우도해녀식당
우도(해양도립공원) 와(과) 가까운 곳 180 망고레이
우도(해양도립공원) 와(과) 가까운 곳 182 로뎀가든
우도(해양도립공원) 와(과) 가까운 곳 192 회양과 국수군
우도(해양도립공원) 와(과) 가까운 곳 208 지미스
우도(해양도립공원) 와(과) 가까운 곳 211 우도동식이
우도(해양도립공원) 와(과) 가까운 곳 251 안녕육지사람
우도(해양도립공원) 와(과) 가까운 곳 313 우도해광식당
우도(해양도립공원) 와(과) 가까운 곳 322 우

In [75]:
food_type_indicator = {'한식':0,'중식':1,'일식':2,'양식':3,'분식':4,'카페/베이커리':5}

In [76]:
save_entire_near_foods = []
cnt=0
for now in attraction.values:    
  now_latitude = now[4]
  now_longitude = now[5]
  now_location = (now_latitude,now_longitude)
  #print('now loc : ',now_location)
  save_near_foods = [[] for _ in range(6)]

  for f in food.values:
    food_id = f[0]
    food_title = f[1]
    food_latitude = f[9]
    food_longitude = f[10]
    food_type = f[5]
    if food_type =='폐업':
      continue
    food_location = (food_latitude,food_longitude)
    
    distance = haversine(now_location,food_location)
    # print(distance)
    if distance<=5.0:
      #now에서 가까운 숙소니까 저장하기 해당하는 숙소 타입의 index를 찾아서 id 추가
      save_near_foods[food_type_indicator[food_type]].append(food_id)

  save_entire_near_foods.append(save_near_foods)

In [77]:
entire_near_foods_df = pd.DataFrame(save_entire_near_foods)

In [78]:
entire_near_foods_df

,0,1,2,3,4,5
0,"[66, 191, 248, 303, 308, 392, 499, 592]",[],[],[1426],[],"[197, 638, 643, 748, 825, 1123, 1346, 1350]"
1,"[134, 137, 176, 182, 192, 211, 313, 338, 487, ...","[546, 974, 1018, 1151, 1340]",[1332],[],[],"[34, 54, 117, 152, 180, 208, 251, 322, 746, 81..."
2,"[9, 19, 49, 86, 90, 102, 110, 134, 142, 153, 1...","[1018, 1151, 1302, 1340]","[379, 486, 1343]","[56, 300, 937, 1234, 1270, 1339, 1355, 1478]","[661, 961]","[34, 152, 180, 187, 208, 267, 294, 296, 322, 3..."
3,"[138, 399, 1049, 1457, 1467]",[],[842],[200],[],"[146, 403, 764, 879, 899, 1327, 1395, 1475, 1572]"
4,"[18, 43, 57, 93, 95, 100, 107, 127, 149, 171, ...","[75, 893]","[96, 290, 312, 406, 437, 451, 517, 777, 854, 1...","[122, 157, 368, 468, 511, 562, 600, 717, 829, ...","[147, 321, 476, 927, 1020, 1379, 1542]","[310, 346, 472, 473, 526, 529, 531, 571, 574, ..."
...,...,...,...,...,...,...
1084,[],[],[],[],[],[]
1085,"[86, 90, 102, 110, 134, 137, 142, 153, 159, 16...","[546, 974, 1018, 1151, 1302, 1340]",[1332],"[56, 937, 1355]",[961],"[34, 54, 117, 152, 180, 187, 208, 251, 267, 29..."
1086,"[4, 50, 92, 155, 163, 524, 615, 713, 772, 1012...","[860, 1402]","[28, 407, 701, 842, 902]","[118, 612, 650, 665, 700, 1080]","[270, 537]","[38, 59, 112, 209, 263, 317, 343, 372, 440, 44..."
1087,"[24, 42, 44, 51, 55, 64, 76, 84, 143, 183, 184...",[577],"[331, 376]","[188, 335, 425, 563, 1334, 1480]",[],"[40, 280, 776, 796, 930, 1090, 1530, 1552, 159..."


In [ ]:
#food_type_indicator = {'한식':0,'중식':1,'일식':2,'양식':3,'분식':4,'카페/베이커리':5}

In [79]:
entire_near_foods_df.columns = ['near_korean','near_chinese','near_japanese','near_western','near_bunsik','near_cafe']
entire_near_foods_df

,near_korean,near_chinese,near_japanese,near_western,near_bunsik,near_cafe
0,"[66, 191, 248, 303, 308, 392, 499, 592]",[],[],[1426],[],"[197, 638, 643, 748, 825, 1123, 1346, 1350]"
1,"[134, 137, 176, 182, 192, 211, 313, 338, 487, ...","[546, 974, 1018, 1151, 1340]",[1332],[],[],"[34, 54, 117, 152, 180, 208, 251, 322, 746, 81..."
2,"[9, 19, 49, 86, 90, 102, 110, 134, 142, 153, 1...","[1018, 1151, 1302, 1340]","[379, 486, 1343]","[56, 300, 937, 1234, 1270, 1339, 1355, 1478]","[661, 961]","[34, 152, 180, 187, 208, 267, 294, 296, 322, 3..."
3,"[138, 399, 1049, 1457, 1467]",[],[842],[200],[],"[146, 403, 764, 879, 899, 1327, 1395, 1475, 1572]"
4,"[18, 43, 57, 93, 95, 100, 107, 127, 149, 171, ...","[75, 893]","[96, 290, 312, 406, 437, 451, 517, 777, 854, 1...","[122, 157, 368, 468, 511, 562, 600, 717, 829, ...","[147, 321, 476, 927, 1020, 1379, 1542]","[310, 346, 472, 473, 526, 529, 531, 571, 574, ..."
...,...,...,...,...,...,...
1084,[],[],[],[],[],[]
1085,"[86, 90, 102, 110, 134, 137, 142, 153, 159, 16...","[546, 974, 1018, 1151, 1302, 1340]",[1332],"[56, 937, 1355]",[961],"[34, 54, 117, 152, 180, 187, 208, 251, 267, 29..."
1086,"[4, 50, 92, 155, 163, 524, 615, 713, 772, 1012...","[860, 1402]","[28, 407, 701, 842, 902]","[118, 612, 650, 665, 700, 1080]","[270, 537]","[38, 59, 112, 209, 263, 317, 343, 372, 440, 44..."
1087,"[24, 42, 44, 51, 55, 64, 76, 84, 143, 183, 184...",[577],"[331, 376]","[188, 335, 425, 563, 1334, 1480]",[],"[40, 280, 776, 796, 930, 1090, 1530, 1552, 159..."


In [80]:
merge_attraction2 = pd.merge(left=merge_attraction,right=entire_near_foods_df,left_index=True,right_index=True,how='inner')

In [82]:
merge_attraction2.head(2)

,id,title,address,introduction,latitude,longitude,dial_num,type,region1,region2,...,near_motel,near_guestHouse,near_hotel,near_camping,near_korean,near_chinese,near_japanese,near_western,near_bunsik,near_cafe
0,1,사려니숲길,제주특별자치도 제주시 조천읍 교래리 산 137-1,"제주 숨은 비경 31, 삼나무 향기에 취하며 걷는 아름답고 청정한 숲길",33.40845,126.63976,064-900-8800,관광지,제주시,조천,...,[],[],[16],[],"[66, 191, 248, 303, 308, 392, 499, 592]",[],[],[1426],[],"[197, 638, 643, 748, 825, 1123, 1346, 1350]"
1,2,우도(해양도립공원),제주특별자치도 제주시 우도면 삼양고수물길 1,소가 누워있는 형상을 하고 있는 제주의 가장 큰 부속섬,33.51949,126.95109,064-728-1527,관광지,섬 속의 섬,우도,...,[],"[186, 249, 504, 554, 755]",[],[],"[134, 137, 176, 182, 192, 211, 313, 338, 487, ...","[546, 974, 1018, 1151, 1340]",[1332],[],[],"[34, 54, 117, 152, 180, 208, 251, 322, 746, 81..."


In [84]:
merge_attraction2 = merge_attraction2[['id','title','near_pension','near_minbak','near_motel','near_guestHouse','near_hotel','near_camping','near_korean','near_chinese','near_japanese','near_western','near_bunsik','near_cafe']]

In [85]:
merge_attraction2.to_csv("/content/drive/MyDrive/yeoreodigm/data_files/09.29_attraction_near_info.csv")

In [86]:
merge_attraction2.head(3)

,id,title,near_pension,near_minbak,near_motel,near_guestHouse,near_hotel,near_camping,near_korean,near_chinese,near_japanese,near_western,near_bunsik,near_cafe
0,1,사려니숲길,"[210, 260]",[],[],[],[16],[],"[66, 191, 248, 303, 308, 392, 499, 592]",[],[],[1426],[],"[197, 638, 643, 748, 825, 1123, 1346, 1350]"
1,2,우도(해양도립공원),"[48, 84, 109, 192, 202, 253, 381, 412, 433, 45...","[312, 531, 587, 644, 651, 664, 679]",[],"[186, 249, 504, 554, 755]",[],[],"[134, 137, 176, 182, 192, 211, 313, 338, 487, ...","[546, 974, 1018, 1151, 1340]",[1332],[],[],"[34, 54, 117, 152, 180, 208, 251, 322, 746, 81..."
2,3,성산일출봉(UNESCO 세계자연유산),"[84, 109, 112, 185, 236, 253, 327, 349, 381, 4...","[367, 465, 679]",[149],"[22, 29, 159, 178, 181, 249, 340, 354, 362, 42...","[7, 34, 38, 40, 43, 50, 51, 107, 317, 397, 619...",[],"[9, 19, 49, 86, 90, 102, 110, 134, 142, 153, 1...","[1018, 1151, 1302, 1340]","[379, 486, 1343]","[56, 300, 937, 1234, 1270, 1339, 1355, 1478]","[661, 961]","[34, 152, 180, 187, 208, 267, 294, 296, 322, 3..."


##DB에 값 입력

In [144]:
db = load_db()
cursor = db.cursor()

In [164]:
db.rollback()

In [167]:
for now in tqdm(merge_attraction2.values):
  id = now[0]
  title = now[1]
  pension,minbak,motel,guestHoust,hotel,camping,korean,chinese,japanese,western,bunsik,cafe = now[2:]
  cursor.execute("insert into near_accomodation(place_id, title, pension, minbak, motel, guest_house, hotel, camping) values(%s, %s, %s, %s, %s, %s, %s, %s)",(id,title,pension,minbak,motel,guestHoust,hotel,camping))
  cursor.execute("insert into near_restaurant(place_id, title, korean,chinese,japanese,western,bunsik,cafe) values(%s, %s, %s, %s, %s, %s, %s, %s)",(id,title,korean,chinese,japanese,western,bunsik,cafe))
  
db.commit()

100%|██████████| 1089/1089 [01:52<00:00,  9.68it/s]
